# Aviation Data Analysis 

put a picture here

## Overview

put some text here

## Business Problem

put a picture here

put some text here

## Data Understanding

put a picture here

put some text here

import libraries here 

import data here

## Aircraft Data

talk about the data here

show the data head and data info here

## Data Preperation

### Data Cleaning

talk about the data cleaning here

clean the data here

## Feature Engineering

talk about the feature engineering

do the feature engineering

save the cleaned dataset as a csv


## Analysis

import any libraries you may need to do the analysis

### Analysis 1

talk about the analysis 

show the analysis

### Analysis 2

talk about the analysis 

show the analysis

### Analysis 3

talk about the analysis

show the analysis

## Conclusions

This analysis leads to three recommendations for ...

    1)conclusion number 1. then another sentence about the conclusion

    2) conclusion number 2. then another sentence about the conclusion

    3) conclusion number 3. then another sentence about the conclusion


### Next Steps

Further Analysis could yield additional insights to further improve operations at blah blah blah

    1) one possible next analysis. another sentence on this 
    2) a second next analysis. another sentence on this 
    3) a third next analysis. another sentence on this